# Model Fairness, Explainability and Drift with Watson OpenScale

This notebook should be run in a Watson Studio project, using Default Python 3.6 runtime environment. It requires service credentials and a Cloud API key to access the following Cloud services:
* Watson Machine Learning
* Watson OpenScale

The notebook will configure several monitors in OpenScale for the German Credit Risk model. The notebook assumes the model has been created/deployed to Watson Machine Learning and that the subscription has been created in Watson OpenScale.

#### Dependency Setup

In [1]:
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1
!pip install --upgrade watson-machine-learning-client | tail -n 1

In [2]:
import pandas as pd
import json
import random

from IPython.utils import io

from watson_machine_learning_client import WatsonMachineLearningAPIClient

from ibm_ai_openscale import APIClient
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
from ibm_ai_openscale.supporting_classes.enums import *

#### Configure Service Credentials

Update the two cells below with your Cloud API Key and your Watson Machine Learning service credentials.

In [3]:
CLOUD_API_KEY="WRSbGHmNSHvy-tLXNfqRvPhnGFfGfx6aLZnVv05h7PsQ"

In [4]:
WML_CREDENTIALS = {
  "apikey": "Rk2Chr1ij8WCQgMThbYME0-o28aa2nwtrAMH7eMspYJP",
  "iam_apikey_description": "Auto-generated for key 1cbdf600-c774-449f-8d7d-3ab37b1a793f",
  "iam_apikey_name": "Service credentials-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/44f657e1cfa9244c30605fcaaa86343a::serviceid:ServiceId-7aa04496-3655-4f42-856e-80957aa89bf6",
  "instance_id": "ead3646b-8ad5-4505-80e7-ff0afafe895b",
  "url": "https://us-south.ml.cloud.ibm.com"
}

#### Model Parameters

We use the same name for the Scikit model and the deployment to WML.

__Ensure that the two parameters match the model / deployment you have previously subscribed__

In [5]:
MODEL_NAME = "Spark German Risk Model"
DEPLOYMENT_NAME = "Spark German Risk Deployment"

#### Load Training Data

Although we have are not creating a model at this point, Watson OpenScale makes use of statistics gathered from the training data for various monitors. 

In [6]:
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/german_credit_data_biased_training.csv  -O german_credit_data_biased_training.csv
    
!ls -lh german_credit_data_biased_training.csv

data_df = pd.read_csv('german_credit_data_biased_training.csv', sep=",", header=0)
#data_df.head()

-rw-r----- 1 dsxuser dsxuser 674K Nov 23 17:17 german_credit_data_biased_training.csv


#### Gather Model Information

In [7]:
wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)
wml_client.repository.list_models()

model_uid = None
wml_models = wml_client.repository.get_details()
for model_in in wml_models['models']['resources']:
    if MODEL_NAME == model_in['entity']['name']:
        model_uid = model_in['metadata']['guid']
        break

deployment_uid = None
deployment = None
scoring_url = None
wml_deployments = wml_client.deployments.get_details()
for deployment_in in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment_in['entity']['name']:
        deployment_uid = deployment_in['metadata']['guid']
        scoring_url = deployment_in['entity']['scoring_url']
        deployment = deployment_in
        break

if model_uid is None:
    print("No model ...")
    
if deployment_uid is None:
    print("No Model deployment...")
    
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))
print("Scoring URL: {}".format(scoring_url))

------------------------------------  -----------------------  ------------------------  ---------
GUID                                  NAME                     CREATED                   FRAMEWORK
e7e5151c-670f-4511-9756-934a0f3ebea3  Spark German Risk Model  2019-11-23T07:04:59.246Z  mllib-2.3
------------------------------------  -----------------------  ------------------------  ---------
Model id: e7e5151c-670f-4511-9756-934a0f3ebea3
Deployment id: 5bea19b8-2ae7-4ce1-8df0-a09148a1485a
Scoring URL: https://us-south.ml.cloud.ibm.com/v3/wml_instances/ead3646b-8ad5-4505-80e7-ff0afafe895b/deployments/5bea19b8-2ae7-4ce1-8df0-a09148a1485a/online


## Fairness and Explainability Monitors

#### Get Watson OpenScale GUID
Each instance of OpenScale has a unique ID. We can get this value using the Cloud API key specified at the beginning of the notebook.

In [8]:
wos_client = None
WOS_GUID = get_instance_guid(api_key=CLOUD_API_KEY)
WOS_CREDENTIALS = {
    "instance_guid": WOS_GUID,
    "apikey": CLOUD_API_KEY,
    "url": "https://api.aiopenscale.cloud.ibm.com"
}

if WOS_GUID is None:
    print('Watson OpenScale GUID NOT FOUND')
else:
    print("Watson OpenScale GUID: {}".format(WOS_GUID))

    
wos_client = APIClient(aios_credentials=WOS_CREDENTIALS)
print("Watson OpenScale Python Client Version: {}".format(wos_client.version))

Watson OpenScale GUID: 0ac203a2-114a-497a-9721-d2dedf99d339
Watson OpenScale Python Client Version: 2.1.19


#### Get subscription

We have previously subscribed Watson OpenScale to our machine learning model. Here we get that subscription.

In [9]:
wos_client.data_mart.subscriptions.list()

subscriptions_uids = wos_client.data_mart.subscriptions.get_uids()
subscription_id = None
for sub in subscriptions_uids:
    if wos_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == MODEL_NAME:
        subscription = wos_client.data_mart.subscriptions.get(sub)
        subscription_id = sub
        break
            
if subscription is None:
    print('Subscription not found.')
    
print("Subscription ID: {}".format(subscription_id))
#print(json.dumps(wos_client.data_mart.subscriptions.get_details(subscription_id),indent=2))

5a11528d-f2ae-4b8f-9f03-08fb90587d1a,Spark German Risk Model,model,ead3646b-8ad5-4505-80e7-ff0afafe895b,2019-11-23T07:07:04.924Z


Subscription ID: 5a11528d-f2ae-4b8f-9f03-08fb90587d1a


### Enable Fairness Monitor

The code below configures fairness monitoring for our model. It turns on monitoring for two features, Sex and Age. In each case, we must specify:
  * Which model feature to monitor
  * One or more **majority** groups, which are values of that feature that we expect to receive a higher percentage of favorable outcomes
  * One or more **minority** groups, which are values of that feature that we expect to receive a higher percentage of unfavorable outcomes
  * The threshold at which we would like OpenScale to display an alert if the fairness measurement falls below (in this case, 95%)

Additionally, we must specify which outcomes from the model are favourable outcomes, and which are unfavourable. We must also provide the number of records OpenScale will use to calculate the fairness score. In this case, OpenScale's fairness monitor will run hourly, but will not calculate a new fairness rating until at least 200 records have been added. Finally, to calculate fairness, OpenScale must perform some calculations on the training data, so we provide the dataframe containing the data.

In [10]:
subscription.fairness_monitoring.enable(
            features=[
                Feature("Sex", majority=['male'], minority=['female'], threshold=0.95),
                Feature("Age", majority=[[26,75]], minority=[[18,25]], threshold=0.95)
            ],
            favourable_classes=['No Risk'],
            unfavourable_classes=['Risk'],
            min_records=200,
            training_data=data_df
        )

### Enable Drift Monitoring

We will now enable drift monitoring that will run when there are 100 records and set a threshold of 10% degradation in performance. The code will then poll until the drift monitoring is configured.

In [11]:
subscription.drift_monitoring.enable(min_records=100, threshold=0.1)

drift_status = None
while drift_status != 'finished':
    drift_details = subscription.drift_monitoring.get_details()
    drift_status = drift_details['parameters']['config_status']['state']
    if drift_status != 'finished':
        print(datetime.utcnow().strftime('%H:%M:%S'), drift_status)
        time.sleep(30)
print("Drift status: {}".format(drift_status)

SyntaxError: unexpected EOF while parsing (<ipython-input-11-3a2e71330b39>, line 10)

### Send Requests to Model 

Now that we have some model monitors enabled, we will send some scoring requests through our model. This next section randomly selects 200 records from the data feed and sends those records to the model for predictions. This is enough to exceed the minimum threshold for records set in the previous section, which allows OpenScale to begin calculating fairness and drift.

In [12]:
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/german_credit_feed.json -O german_credit_feed.json
!ls -lh german_credit_feed.json
   
with open('german_credit_feed.json', 'r') as scoring_file:
    scoring_data = json.load(scoring_file)

fields = scoring_data['fields']
values = []
for _ in range(200):
    values.append(random.choice(scoring_data['values']))
payload_scoring = {"fields": fields, "values": values}

scoring_response = wml_client.deployments.score(scoring_url, payload_scoring)

print('Number of scoring result:', len(scoring_response['values']))
time.sleep(10)
print('Number of records in payload table: ', subscription.payload_logging.get_records_count())

-rw-r----- 1 dsxuser dsxuser 3.0M Nov 23 17:19 german_credit_feed.json
Number of scoring result: 200
Number of records in payload table:  208


__Note:__ The number of records in the payload table below should be 208 (the 200 scoring requests made above and the initial 8 scoring requests sent prior to monitor configuration). 

In [13]:
print('Number of records in payload table: ', subscription.payload_logging.get_records_count())
#subscription.payload_logging.show_table(limit=20)

Number of records in payload table:  208


### Run fairness monitor

Kick off a fairness monitor run on current data. The monitor runs hourly, but can be manually initiated using the Python client, the REST API, or the graphical user interface.

In [14]:
run_details = subscription.fairness_monitoring.run(background_mode=False)




 Counting bias for deployment_uid=5bea19b8-2ae7-4ce1-8df0-a09148a1485a 




RUNNING......
FINISHED

---------------------------
 Successfully finished run 
---------------------------




In [15]:
time.sleep(10)
subscription.fairness_monitoring.show_table()

2019-11-23 17:31:31.668657+00:00,Sex,female,True,0.907,68.5,ead3646b-8ad5-4505-80e7-ff0afafe895b,5a11528d-f2ae-4b8f-9f03-08fb90587d1a,5a11528d-f2ae-4b8f-9f03-08fb90587d1a,5bea19b8-2ae7-4ce1-8df0-a09148a1485a,
2019-11-23 17:31:31.668657+00:00,Age,"[18, 25]",False,1.087,81.0,ead3646b-8ad5-4505-80e7-ff0afafe895b,5a11528d-f2ae-4b8f-9f03-08fb90587d1a,5a11528d-f2ae-4b8f-9f03-08fb90587d1a,5bea19b8-2ae7-4ce1-8df0-a09148a1485a,


### Run drift monitor

Kick off a drift monitor run on current data. The monitor runs every hour to compare transaction data against training data patterns. We can be manually initiate this using the Python client, the REST API, or the graphical user interface.

In [18]:
drift_run_details = subscription.drift_monitoring.run(background_mode=False)

KeyError: 'parameters'

In [19]:
time.sleep(10)
subscription.drift_monitoring.get_table_content()

,ts,id,measurement_id,value,lower limit,upper limit,tags,binding_id,subscription_id,deployment_id


## Next steps

We can now monitor the model or configure some of the other monitors (quality, fairness, explainability, drift, etc) using either the UI or through a python client.

__Return to the workshop instruction book.__


## Credits

This notebook was adapted from the following sources:

* [Monitor Models Code Pattern](https://github.com/IBM/monitor-wml-model-with-watson-openscale)
* [OpenScale Labs](https://github.com/pmservice/OpenScale-Labs)
* [OpenScale Tutorials](https://github.com/pmservice/ai-openscale-tutorials)

#### Original Authors
* Eric Martens, is a technical specialist having expertise in analysis and description of business processes, and their translation into functional and non-functional IT requirements. He acts as the interpreter between the worlds of IT and business.
* Lukasz Cmielowski, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.